In [1]:
# import os
# os.chdir('home')
# os.getcwd()

In [2]:
import yaml
import json
import os
from ultralytics import YOLO



In [3]:
# def process_metadata(metadata_path, output_dir):
#     with open(metadata_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             data = json.loads(line)
#             file_name = data['file_name']
#             gt_parse = json.loads(data['ground_truth'])['gt_parse']
            
#             # YOLO 형식으로 bounding box 정보 변환
#             yolo_boxes = []
#             for class_name, box in gt_parse['roi_yolo'].items():
#                 class_id = 0 if class_name == 'note' else 1  # 'note'는 0, 'layouts'는 1
#                 yolo_boxes.append(f"{class_id} {' '.join(map(str, box))}")
            
#             # YOLO 형식의 레이블 파일 생성
#             label_path = os.path.join(output_dir, os.path.splitext(file_name)[0] + '.txt')
#             with open(label_path, 'w') as label_file:
#                 label_file.write('\n'.join(yolo_boxes))
# # 메타데이터 처리 및 YOLO 형식 레이블 생성
# metadata_path = 'data/rawdata/metadata.jsonl'
# output_dir = 'data/labels'
# os.makedirs(output_dir, exist_ok=True)
# process_metadata(metadata_path, output_dir)

In [4]:
# 설정 파일 경로
train_config_path = 'configs/train_config.yaml'
dataset_config_path = 'configs/dataset.yaml'

# 학습 설정 로드
with open(train_config_path, 'r') as file:
    train_config = yaml.safe_load(file)

# 모델 생성
model = YOLO('models/yolov8s.pt')

In [5]:
import ultralytics
print(ultralytics.checks())

Ultralytics YOLOv8.2.78  Python-3.9.19 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16375MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 53.1/111.3 GB disk)
None


In [6]:
import torch
import torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA is available: {torch.cuda.is_available()}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

# CUDA 텐서 연산 테스트
x = torch.rand(5, 3).cuda()
y = torch.rand(5, 3).cuda()
print(torch.mm(x, y.t()))

# torchvision CUDA 기능 테스트
boxes = torch.rand(10, 4).cuda()
scores = torch.rand(10).cuda()
print(torchvision.ops.nms(boxes, scores, 0.5))

PyTorch version: 2.4.0+cu121
Torchvision version: 0.19.0+cu121
CUDA is available: True
cuDNN version: 90100
Current CUDA device: 0
CUDA device name: NVIDIA GeForce RTX 4070 Ti SUPER
tensor([[0.4946, 0.7269, 0.6952, 0.3635, 0.8844],
        [0.4142, 0.3740, 0.2865, 0.0965, 0.4679],
        [0.3643, 0.4135, 0.4486, 0.2981, 0.5851],
        [0.5941, 0.5006, 0.5522, 0.3927, 0.7853],
        [0.2454, 0.5714, 0.4898, 0.1819, 0.5829]], device='cuda:0')
tensor([9, 6, 5, 4, 3, 8, 2, 7, 0, 1], device='cuda:0')


In [7]:
# 학습 시작
results = model.train(
    data=dataset_config_path,
    
    **train_config
)

Ultralytics YOLOv8.2.78  Python-3.9.19 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16375MiB)
engine\trainer: task=detect, mode=train, model=models/yolov8s.pt, data=configs/dataset.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs/train, name=yolo_training_test, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

c:\ProgramData\miniconda3\envs\yolov8\lib\site-packages\ultralytics\engine\trainer.py:271: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning D:\workspace\SAHI_OCR\data\note\train\labels.cache... 8000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8000/8000 [00:00<?, ?it/s]
val: Scanning D:\workspace\SAHI_OCR\data\note\val\labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


Plotting labels to runs\train\yolo_training_test\labels.jpg... 
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs\train\yolo_training_test
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      9.07G     0.9495     0.9597      1.418         47       1024: 100%|██████████| 500/500 [02:31<00:00,  3.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:08<00:00,  3.60it/s]


                   all       1000       2000      0.886      0.904      0.925      0.647

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      9.14G     0.1945       0.13     0.8847         31       1024: 100%|██████████| 500/500 [01:53<00:00,  4.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:07<00:00,  4.07it/s]

                   all       1000       2000          1          1      0.995      0.991



100 epochs completed in 7.235 hours.
Optimizer stripped from runs\train\yolo_training_test\weights\last.pt, 22.6MB
Optimizer stripped from runs\train\yolo_training_test\weights\best.pt, 22.6MB

Validating runs\train\yolo_training_test\weights\best.pt...
Ultralytics YOLOv8.2.78  Python-3.9.19 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16375MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:19<00:00,  1.62it/s]


                   all       1000       2000          1          1      0.995      0.991
                  note       1000       1000          1          1      0.995      0.992
               layouts       1000       1000          1          1      0.995      0.989
Speed: 0.7ms preprocess, 3.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\train\yolo_training_test
